## Perceptron to perform OR operation

This notebook implements a basic perceptron to perform logical OR operation. However, this strategy can be followed for any other logical operation as well, because of the simplicity of the problem. 

In [1]:
# Numpy for some math utility functions
import numpy as np

In [2]:
def sigmoid(x):
    '''
    Function to compute sigmoid
    '''
    return 1 / (1 + np.exp(-x))

def cross_entropy_loss(y_hat, y):
    '''
    Function to compute cross entropy loss
    '''
    if y == 1:
        return -np.log(y_hat)
    else:
        return -np.log(1 - y_hat)

In [4]:
class BasicPerceptron:
    '''
    Implements an basic artificial neuron, which is good enough for logical gates. 
    Basic Operation --> w0*x1 + w1*x2 + b 
    w --> weights 
    b --> biases
    x --> input
    
    Since OR Game requires 2 inputs it is x1 and x2 given as input. 
    '''
    def __init__(self, init_method="random", thresh=0.5):
        # initialize the weights and biases
        self.W_0 = None
        self.W_1 = None
        self.b = None
        
        self._initialize(init_method)
        
    def _initialize(self, method):
        '''
        Helper function for weight initialization, supports 3 methods
        "random" - Random weight initilization (close to zero)
        "zeros"  - initializes with zeros
        "or_gate"- initializes weights for OR Gate (manually computed)
                   Formula = 2*x1 + 2*x2 + (-1)
                   Sigmoid = g(x)
                   When x=0,y=0 --> 2*0 + 2*0 + (-1) = g(-1) = 0
                   When x=0,y=1 --> 2*0 + 2*1 + (-1) = g(1) = 1
                   When x=1,y=0 --> 2*1 + 2*0 + (-1) = g(1) = 1
                   When x=1,y=1 --> 2*1 + 2*1 + (-1) = g(3) = 1
                    
        ''' 
        if method == "random":
            self.W_0 = (np.random.randn(1)*0.1)[0]
            self.W_1 = (np.random.randn(1)*0.1)[0]
            self.b = (np.random.randn(1)*0.1)[0]
        elif method == "or_gate":
            self.W_0 = 2
            self.W_1 = 2
            self.b = -1
        elif method == "zeros":
            self.W_0 = 0.0
            self.W_1 = 0.0
            self.b = 0.0
        else:
            raise("Initialization method unknown. Choose one of [random, or_gate, zeros]")
        
    def feed_forward(self, X):
        '''
        Performs one forward pass through the network to obtain the prediction
        ''' 
        x1, x2 = X
        return sigmoid(self.W_0 * x1 + self.W_1 * x2 + self.b)    
    
    def train(self, train_samples, learning_rate, epochs):
        '''
        Train function - Since we are training just a single neuron,
        computing derivative and chain rule to update weights isn't necessary.
        
        if W and b are weight matrices, the operation performed here is,
           W = W + lr * (y - y_hat) * X
           b = b + lr * (y - y_hat)
           
        This simple strategy is good enough for this problem
        '''
        for ep in range(epochs):
            losses = []
            for _, (X, y) in enumerate(train_samples):
                pred = self.feed_forward(X)
                losses.append(cross_entropy_loss(pred, y))
                for i in range(len(X)):
                    self.W_0 += learning_rate * (y - pred) * X[i]
                    self.W_1 += learning_rate * (y - pred) * X[i]
                self.b += learning_rate * (y - pred)
                    
            print(f"Epoch: {ep} --> Loss: {np.mean(losses)}")
            
    def predict(self, inputs, return_prob=False):
        '''
        Predicts outputs for the given array of input
        '''
        preds = []
        
        for inp in inputs:
            prob = self.feed_forward(inp)
            if return_prob:
                preds.append(prob)
                continue
            preds.append(int(prob > 0.5))
            
        return preds

## With Pretuned values
As mentioned in the function description above, first lets test out the manually tuned values. 

> **Note: This is not the only combination of weights that result in similar behaviour. w0 = 20, w1 =20 and b =-10 should also work the same way.**  

In [5]:
nn_ = BasicPerceptron(init_method="or_gate")

in_data = [[0, 0], [0, 1], [1, 0], [1, 1]] 

preds = nn_.predict(in_data, return_prob=False)

for i, inp in enumerate(preds):
    print(f"In: {in_data[i]} --> out: {preds[i]}")

In: [0, 0] --> out: 0
In: [0, 1] --> out: 1
In: [1, 0] --> out: 1
In: [1, 1] --> out: 1


## Training the Neural Network from scratch
Now the NN is randomly initialized and trained to make it learn the Logical OR operation itself. 

In [6]:
nn = BasicPerceptron(init_method="random")

out_data = [0, 1, 1, 1]

train_samples = list(zip(in_data, out_data))

def generate_data(array, size):
    train_data = []
    for i in range(size):
        train_data.append(array[np.random.choice(range(len(array)))])
    return train_data

In [7]:
print(nn.W_0, nn.W_1, nn.b)

-0.018581960812328505 0.023660940350937687 -0.266212301816733


In [8]:
train_data = generate_data(train_samples, 500)

In [9]:
nn.train(train_data, learning_rate=0.1, epochs=10)

Epoch: 0 --> Loss: 0.1934103226842079
Epoch: 1 --> Loss: 0.07007885363209766
Epoch: 2 --> Loss: 0.042375717641964135
Epoch: 3 --> Loss: 0.030199974640152227
Epoch: 4 --> Loss: 0.023401224407746297
Epoch: 5 --> Loss: 0.019076171972197683
Epoch: 6 --> Loss: 0.016088096896181722
Epoch: 7 --> Loss: 0.013902551669624827
Epoch: 8 --> Loss: 0.012235757729789108
Epoch: 9 --> Loss: 0.010923302495634982


In [10]:
preds = nn.predict(in_data, return_prob=True)
   
for i, inp in enumerate(preds):
    print(f"in: {in_data[i]} --> prob: {round(preds[i], 4)} --> out: {int(preds[i] > 0.5)}")

in: [0, 0] --> prob: 0.0252 --> out: 0
in: [0, 1] --> prob: 0.9922 --> out: 1
in: [1, 0] --> prob: 0.9918 --> out: 1
in: [1, 1] --> prob: 1.0 --> out: 1


In [11]:
test_data = generate_data(train_samples, 100)

X_test = list(zip(*test_data))[0]
y_test = list(zip(*test_data))[1]

In [12]:
preds = nn.predict(X_test)

def compute_accuracy(y_hat, y):
    correct_counter = 0
    for i in range(len(y)):
        if y_hat[i] == y[i]:
            correct_counter += 1
    return correct_counter / len(y)
    
acc = compute_accuracy(preds, y_test)
print(f"Test Accuracy: {acc}")

Test Accuracy: 1.0


As you can see our NN now becomes really good at this simple task and gets 100% accuracy. Which in this case is not an anomaly.

**Contributor: [abhinand05](www.github.com/abhinand5)**